In [ ]:
# # Auto reload
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from transformer_lens import HookedTransformer

from sparse_autoencoder.autoencoder.model import SparseAutoencoder
from sparse_autoencoder.dataset.dataloader import (
    collate_neel_c4_tokenized,
    create_dataloader,
)
from sparse_autoencoder.train.train import pipeline

In [ ]:
src_model = HookedTransformer.from_pretrained("NeelNanda/GELU_1L512W_C4_Code")
n_input_features = src_model.cfg.d_mlp

autoencoder = SparseAutoencoder(
    n_input_features=n_input_features,
    n_learned_features=n_input_features * 8,
    geometric_median_dataset=torch.zeros(n_input_features),
)
autoencoder

In [ ]:
dataloader = create_dataloader(
    "NeelNanda/c4-code-tokenized-2b",
    collate_fn=collate_neel_c4_tokenized,
    shuffle_buffer_size=1000,
    batch_size=2,
    num_workers=1,
)

In [ ]:
pipeline(
    src_model=src_model,
    autoencoder=autoencoder,
    activation_hook_point="blocks.0.mlp.hook_post",
    prompts_dataloader=dataloader,
)